In [27]:
import sympy as sym

In [28]:
t = sym.Symbol('t')

I = sym.Symbol('I')
m = sym.Symbol('m')
m_inf = sym.Symbol('m_inf')
p = sym.Symbol('p')
prestep_V = sym.Symbol('prestep_V')
step_V = sym.Symbol('step_V')
a = sym.Symbol('a')
b = sym.Symbol('b')

g = sym.Symbol('g')
E_rev = sym.Symbol('E_rev')
M_ma = sym.Symbol('M_ma')
M_mb = sym.Symbol('M_mb')
V_2ma = sym.Symbol('V_2ma')
V_2mb = sym.Symbol('V_2mb')
s_ma = sym.Symbol('s_ma')
s_mb = sym.Symbol('s_mb')

In [36]:
# H-H model
m_inf = 1 / (1 + M_mb/M_ma * ((1+sym.exp((prestep_V-V_2ma)/s_ma)) / (1+sym.exp((prestep_V-V_2mb)/s_mb))))

a = M_ma / (1 + sym.exp((step_V - V_2ma) / s_ma))
b = M_mb / (1 + sym.exp((step_V - V_2mb) / s_mb))

m = (a - (a-(a+b)*m_inf)*sym.exp(-t*(a+b))) / (a + b)
I = g * m ** 4 * (step_V - E_rev)

In [37]:
dI_dg = sym.diff(I, g)
dI_dE_rev = sym.diff(I, E_rev)
dI_dM_ma = sym.diff(I, M_ma)
dI_dM_mb = sym.diff(I, M_mb)
dI_dV_2ma = sym.diff(I, V_2ma)
dI_dV_2mb = sym.diff(I, V_2mb)
dI_ds_ma = sym.diff(I, s_ma)
dI_ds_mb = sym.diff(I, s_mb)

In [39]:
(s_ma / I) * dI_ds_ma

s_ma*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))**4*(-4*M_ma*g*(-E_rev + step_V)*(-V_2ma + step_V)*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))/(1 + M_mb*(exp((-V_2ma + prestep_V)/s_ma) + 1)/(M_ma*(exp((-V_2mb + prestep_V)/s_mb) + 1))))*exp(-t*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))))**4*exp((-V_2ma + step_V)/s_ma)/(s_ma**2*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))**5*(exp((-V_2ma + step_V)/s_ma) + 1)**2) + g*(-E_rev + step_V)*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) - (M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + step_V)/s_mb) + 1))/(1 + M_mb*(exp((-V_2ma + prestep_V)/s_ma) + 1)/(M_ma*(exp((-V_2mb + prestep_V)/s_mb) + 1))))*exp(-t*(M_ma/(exp((-V_2ma + step_V)/s_ma) + 1) + M_mb/(exp((-V_2mb + 

In [10]:
m_inf.subs({prestep_V:-100}).evalf()

1/(1.0 + M_mb*(exp((-V_2ma - 100)/s_ma) + 1.0)/(M_ma*(exp((-V_2mb - 100)/s_mb) + 1.0)))

In [ ]:
dI_dg.subs({step_V: 20, m_inf: m_inf})